In [1]:
include("functions.jl")

update_draft (generic function with 1 method)

In [12]:
#TODO: August 24 ADP data
#TODO: figure out how to rate the players that are never "starters"

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=0);

In [3]:
function dynasty_sf_model(team, pv, drafted, temp=[])
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    T = 10 # num teams
    P = size(data)[1] # num_players
    Y = 10 # num_years projecting
    R = 26 # roster size

    alpha = 3 # starter weight (bench = 1, starter = 1+alpha)
    
    min_qbs = 3
    max_qbs = 5
    min_rbs = 7
    max_rbs = 11
    min_wrs = 7
    max_wrs = 11
    min_tes = 2
    max_tes = 4

    # this year and next year as of now
    starting_qbs_now = 2
    starting_rbs_now = 3
    starting_wrs_now = 3
    starting_flex_now = 8
    starting_tes_now = 1

    #years 3-5
    starting_qbs_mid = 2
    starting_rbs_mid = 2
    starting_wrs_mid = 2
    starting_flex_mid = 6
    starting_tes_mid = 1

    #years 6-7
    starting_qbs_fut = 1
    starting_flex_fut = 4
    starting_tes_fut = 0


    opps = players_drafted_opps(team, T)

    num_picked = sum(opps[i, "Drafted"] for i in 1:P) + length(drafted[team])

    picks = [team, (2*T+1)-team]
    for i in 3:R
        push!(picks, picks[i-2]+(2*T))
    end

    # VARIABLES
    @variable(model, x[i = 1:P], Bin) # whether or not player was drafted by desired team
    @variable(model, y[i = 1:P, j = 1:Y], Bin) # whether or not player is "starting" in year j on your roster

    # OBJECTIVE FUNCTION
    @objective(model, Max, sum(data[i, j+5]*(x[i]+alpha*y[i,j])*(pv^(j-1)) for i in 1:P, j in 1:Y))

    # CONSTRAINTS
    # roster must contain already drafted num_players
    @constraint(model, [i = drafted[team]], x[player_to_index[i]] == 1)

    # roster must not contain any opps
    @constraint(model, [i=1:P], x[i]+opps[i,"Drafted"] <= 1)

    # roster must not contain any temps
    @constraint(model, [i=1:length(temp)], x[player_to_index[temp[i]]] == 0)

    # selecting players in y only if on roster
    @constraint(model, [i = 1:P, j = 1:Y], y[i,j] <= x[i])

    # roster must contain R players (no kicker/DEF)
    @constraint(model, sum(x[i] for i in 1:P) <= R)

    # positional constraints
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) >= min_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) <= max_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) >= min_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) <= max_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) >= min_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) <= max_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) >= min_tes)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) <= max_tes)

    # starting constraints
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in qbs) == starting_qbs_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in rbs) >= starting_rbs_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in wrs) >= starting_wrs_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in flex) == starting_flex_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in tes) == starting_tes_now)

    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in qbs) == starting_qbs_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in rbs) >= starting_rbs_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in wrs) >= starting_wrs_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in flex) == starting_flex_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in tes) == starting_tes_mid)

    @constraint(model, [j = 6:7], sum(y[player_to_index[i],j] for i in qbs) == starting_qbs_fut)
    @constraint(model, [j = 6:7], sum(y[player_to_index[i],j] for i in flex) == starting_flex_fut)
    @constraint(model, [j = 6:7], sum(y[player_to_index[i],j] for i in tes) == starting_tes_fut)

    @constraint(model, [j = 8:10], sum(y[i,j] for i in 1:P) >= 2)


    # DRAFT POSITION CONSTRAINTS
    #at least 1 pick from set of players after each round
    first_avail = 1
    while opps[first_avail, "Drafted"] == 1
        first_avail += 1
    end

    @constraint(model, sum(x[i] for i in first_avail:P) >= R-length(drafted[team]))
    
    for pick in length(drafted[team])+2:R
        @constraint(model, sum(x[i] for i in (picks[pick]-sum(opps[j, "Drafted"] for j in picks[pick]:P)):P) >= R+1-pick)
    end

    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)

    roster = DataFrame(Name = String[], Position = String[], Pick = Int64[], ADP = Float64[])
    count = 1
    for i in drafted[team]
        if i == "QB" || i == "RB" || i == "WR" || i == "TE"
            push!(roster, [i, i, picks[count], 500])
            continue
        end
        push!(roster, [data[player_to_index[i], "Name"], data[player_to_index[i], "Position"], picks[count], data[player_to_index[i], "Dyn_SF"]])
        count += 1
    end
    for i in 1:P
        if value.(x[i]) == 1
            if data[i, "Name"] in drafted[team]
                continue
            else
                push!(roster, [data[i, "Name"], data[i, "Position"], picks[count], data[i, "Dyn_SF"]])
                count += 1
            end
        end
    end
    next_pick = ""
    for i in 1:P
        if value.(x[i]) == 1
            if data[i, "Name"] in drafted[team]
                continue
            else
                next_pick = data[i, "Name"]
                break
            end
        end
    end
    println("Total VORP: " * string(round(objective_value(model), digits = 1)))
    return next_pick, roster
end

dynasty_sf_model (generic function with 2 methods)

In [4]:
data, player_to_index, qbs, rbs, wrs, tes, flex, names = initialize_data()

(491×27 DataFrame
 Row │ FirstName  LastName     Dyn_SF    RedraftHalfPPR  Position  Y0_VORP   Y ⋯
     │ String15   String31     Float64?  Float64?        String3   Float64   F ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ Patrick    Mahomes           1.1            14.8  QB         9.06229  6 ⋯
   2 │ Josh       Allen             2.7            20.4  QB         8.31693  4
   3 │ Jalen      Hurts             3.4            24.3  QB         8.0804   7
   4 │ Joe        Burrow            4.0            35.7  QB         6.92221  6
   5 │ Justin     Jefferson         5.9             1.5  WR        10.5095   8 ⋯
   6 │ Justin     Herbert           6.6            44.5  QB         4.64313  3
   7 │ Ja'Marr    Chase             7.9             3.0  WR         9.97006  6
   8 │ Lamar      Jackson           8.7            34.1  QB         7.18824  5
  ⋮  │     ⋮           ⋮          ⋮            ⋮            ⋮         ⋮        ⋱
 485 │ Tyler      Davi

In [5]:
drafted = update_draft("991541422920695808")
pv = 0.95
first_choice, roster = dynasty_sf_model(5, pv, drafted);
second_choice, r2 = dynasty_sf_model(5, pv, drafted, [first_choice]);
third_choice, r3 = dynasty_sf_model(5, pv, drafted, [first_choice, second_choice])
fourth_choice, r4 = dynasty_sf_model(5, pv, drafted, [first_choice, second_choice, third_choice])
fifth_choice, r5 = dynasty_sf_model(5, pv, drafted, [first_choice, second_choice, third_choice, fourth_choice])
sixth_choice, r6 = dynasty_sf_model(5, pv, drafted, [first_choice, second_choice, third_choice, fourth_choice, fifth_choice])
seventh_choice, r7 = dynasty_sf_model(5, pv, drafted, [first_choice, second_choice, third_choice, fourth_choice, fifth_choice, sixth_choice])

first_choice, second_choice, third_choice, fourth_choice, fifth_choice, sixth_choice, seventh_choice

KeyError: KeyError: key "RB" not found

In [7]:
r7

UndefVarError: UndefVarError: r7 not defined

In [6]:
view_roster(5)

,Position,Player
,String,String
1,QB1,Bryce Young
2,QB2,Jordan Love
3,RB1,Breece Hall
4,RB2,Joe Mixon
5,WR1,Justin Jefferson
6,WR2,Cooper Kupp
7,TE,Travis Kelce
8,FLEX,Deebo Samuel
9,FLEX,D'Andre Swift


In [33]:
drafted = update_draft("991541422920695808")

KeyError: KeyError: key 9 not found

In [38]:
view_roster(5)

,Position,Player
,String,String
1,QB1,Joe Burrow
2,QB2,Daniel Jones
3,RB1,Tony Pollard
4,RB2,Joe Mixon
5,WR1,Cooper Kupp
6,WR2,Davante Adams
7,TE,Travis Kelce
8,FLEX,Chris Godwin
9,FLEX,Isiah Pacheco


In [18]:
roster

,Name,Position,Pick,ADP
,String,String,Int64,Float64
1,Justin Jefferson,WR,5,5.5
2,Travis Kelce,TE,16,20.3
3,Cooper Kupp,WR,25,31.3
4,Josh Jacobs,RB,36,39.1
5,Tony Pollard,RB,45,50.4
6,Jordan Addison,WR,56,55.8
7,Joe Mixon,RB,65,72.0
8,Marquise Brown,WR,76,87.7
9,Amari Cooper,WR,85,79.9


REDRAFT MODEL

In [24]:
function redraft_model(team, num_teams, drafted, temp=[])
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    T = num_teams # num teams
    P = size(data)[1] # num_players
    R = 16 # roster size

    min_qbs = 1
    max_qbs = 2
    min_rbs = 4
    max_rbs = 8
    min_wrs = 4
    max_wrs = 8
    min_tes = 1
    max_tes = 3

    starting_qbs_now = 1
    starting_rbs_now = 2
    starting_wrs_now = 2
    starting_flex_now = 7
    starting_tes_now = 1

    alpha = 3 # weight on starting players


    opps = players_drafted_opps(team, T)

    num_picked = sum(opps[i, "Drafted"] for i in 1:P) + length(drafted[team])

    picks = [team, (2*T+1)-team]
    for i in 3:R
        push!(picks, picks[i-2]+(2*T))
    end

    # VARIABLES
    @variable(model, x[i = 1:P], Bin) # whether or not player was drafted by desired team
    @variable(model, y[i = 1:P], Bin) # whether or not player is "starting" in year j on your roster

    # OBJECTIVE FUNCTION
    @objective(model, Max, sum(data[i, "Y0_VORP"]*(x[i]+alpha*y[i]) for i in 1:P))

    # CONSTRAINTS
    # roster must contain already drafted num_players
    @constraint(model, [i = drafted[team]], x[player_to_index[i]] == 1)

    # roster must not contain any opps
    @constraint(model, [i=1:P], x[i]+opps[i,"Drafted"] <= 1)

    # selecting players to start only if on roster
    @constraint(model, [i = 1:P], y[i] <= x[i])

    # roster must not contain any temps
    @constraint(model, [i=1:length(temp)], x[player_to_index[temp[i]]] == 0)

    # roster must contain 26 players (no kicker/DEF)
    @constraint(model, sum(x[i] for i in 1:P) <= R)

    # positional constraints
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) >= min_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) <= max_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) >= min_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) <= max_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) >= min_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) <= max_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) >= min_tes)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) <= max_tes)

    @constraint(model, sum(y[player_to_index[i]] for i in qbs) == starting_qbs_now)
    @constraint(model, sum(y[player_to_index[i]] for i in rbs) >= starting_rbs_now)
    @constraint(model, sum(y[player_to_index[i]] for i in wrs) >= starting_wrs_now)
    @constraint(model, sum(y[player_to_index[i]] for i in flex) == starting_flex_now)
    @constraint(model, sum(y[player_to_index[i]] for i in tes) == starting_tes_now)


    # DRAFT POSITION CONSTRAINTS

    first_avail = 1
    while opps[first_avail, "Drafted"] == 1
        first_avail += 1
    end

    @constraint(model, sum(x[i] for i in first_avail:P) >= R-length(drafted[team]))

    #at least 1 pick from set of players after each round
    for pick in length(drafted[team])+1:R
        @constraint(model, sum(x[i] for i in (picks[pick]-sum(opps[j, "Drafted"] for j in picks[pick]:P)):P) >= R+1-pick)
    end

    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)

    roster = DataFrame(Name = String[], Position = String[], Pick = Int64[], ADP = Float64[])
    count = 1
    for i in drafted[team]
        if i == "QB" || i == "RB" || i == "WR" || i == "TE"
            push!(roster, [i, i, picks[count], 500])
            continue
        end
        push!(roster, [data[player_to_index[i], "Name"], data[player_to_index[i], "Position"], picks[count], data[player_to_index[i], "RedraftHalfPPR"]])
        count += 1
    end
    for i in 1:P
        if value.(x[i]) == 1
            if data[i, "Name"] in drafted[team]
                continue
            else
                push!(roster, [data[i, "Name"], data[i, "Position"], picks[count], data[i, "RedraftHalfPPR"]])
                count += 1
            end
        end
    end
    next_pick = ""
    for i in 1:P
        if value.(x[i]) == 1
            if data[i, "Name"] in drafted[team]
                continue
            else
                next_pick = data[i, "Name"]
                break
            end
        end
    end
    println("Total VORP: " * string(round(objective_value(model), digits = 1)))
    return next_pick, roster
end

redraft_model (generic function with 2 methods)

In [31]:
data, player_to_index, qbs, rbs, wrs, tes, flex, names = initialize_data(false)

(491×27 DataFrame
 Row │ FirstName  LastName     Dyn_SF    RedraftHalfPPR  Position  Y0_VORP   Y ⋯
     │ String15   String31     Float64?  Float64?        String3   Float64   F ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ Justin     Jefferson         5.9             1.5  WR        10.5095   8 ⋯
   2 │ Christian  McCaffrey        13.9             2.2  RB         7.90528  6
   3 │ Ja'Marr    Chase             7.9             3.0  WR         9.97006  6
   4 │ Austin     Ekeler           28.9             4.0  RB         8.14655  5
   5 │ Travis     Kelce            18.0             5.7  TE         7.85383  6 ⋯
   6 │ Tyreek     Hill             23.5             6.9  WR         8.65459  6
   7 │ Cooper     Kupp             32.4             7.5  WR        10.5522   7
   8 │ Bijan      Robinson         10.5             8.0  RB         6.99868  8
  ⋮  │     ⋮           ⋮          ⋮            ⋮            ⋮         ⋮        ⋱
 485 │ Tyler      Davi

In [34]:
drafted = reset_draft(8)

Dict{Any, Any} with 8 entries:
  5 => Any[]
  4 => Any[]
  6 => Any[]
  7 => Any[]
  2 => Any[]
  8 => Any[]
  3 => Any[]
  1 => Any[]

In [35]:
first_choice_r, roster_r = redraft_model(4, 8, drafted);
second_choice_r, r2_r = redraft_model(4, 8, drafted, [first_choice_r]);
third_choice_r, r3_r = redraft_model(4, 8, drafted, [first_choice_r, second_choice_r])
fourth_choice_r, r4_r = redraft_model(4, 8, drafted, [first_choice_r, second_choice_r, third_choice_r])
fifth_choice_r, r5_r = redraft_model(4, 8, drafted, [first_choice_r, second_choice_r, third_choice_r, fourth_choice_r])
sixth_choice_r, r6_r = redraft_model(4, 8, drafted, [first_choice_r, second_choice_r, third_choice_r, fourth_choice_r, fifth_choice_r])
seventh_choice_r, r7_r = redraft_model(4, 8, drafted, [first_choice_r, second_choice_r, third_choice_r, fourth_choice_r, fifth_choice_r, sixth_choice_r])
println(first_choice_r)
println(second_choice_r)
println(third_choice_r)
println(fourth_choice_r)
println(fifth_choice_r)
println(sixth_choice_r)
println(seventh_choice_r)

Total VORP: 237.1
Total VORP: 230.8
Total VORP: 229.5
Total VORP: 227.5


Total VORP: 223.6
Total VORP: 223.4
Total VORP: 223.2
Cooper Kupp
Travis Kelce
Tyreek Hill
Austin Ekeler
Patrick Mahomes
CeeDee Lamb
Saquon Barkley


In [26]:
roster_r

,Name,Position,Pick,ADP
,String,String,Int64,Float64
1,Cooper Kupp,WR,4,7.5
2,Davante Adams,WR,13,16.6
3,Josh Allen,QB,20,20.4
4,Breece Hall,RB,29,33.4
5,Aaron Jones,RB,36,42.0
6,TJ Hockenson,TE,45,46.2
7,James Conner,RB,52,61.6
8,Chris Godwin,WR,61,63.0
9,Alvin Kamara,RB,68,68.0
